In [1]:
import numpy as np
import torch
from transformers import AutoTokenizer
import datasets
from utils_generator import GenerationModel


In [2]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")

model = GenerationModel("Helsinki-NLP/opus-mt-de-en")

/Users/gauthierroy/opt/anaconda3/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [3]:
data_de= datasets.load_dataset('Helsinki-NLP/tatoeba_mt','deu-eng',split=datasets.Split.TEST, verification_mode= datasets.VerificationMode.NO_CHECKS)

data_fr = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','eng-fra',split=datasets.Split.TEST, verification_mode= datasets.VerificationMode.NO_CHECKS)

data_nld = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','eng-nld',split=datasets.Split.TEST, verification_mode= datasets.VerificationMode.NO_CHECKS)

Found cached dataset tatoeba_mt (/Users/gauthierroy/.cache/huggingface/datasets/Helsinki-NLP___tatoeba_mt/deu-eng/0.0.0/01e819f3f64a772a2ca70949061d295d3a2dc99d05183fe4776a3be23f75f619)
Found cached dataset tatoeba_mt (/Users/gauthierroy/.cache/huggingface/datasets/Helsinki-NLP___tatoeba_mt/eng-fra/0.0.0/01e819f3f64a772a2ca70949061d295d3a2dc99d05183fe4776a3be23f75f619)
Found cached dataset tatoeba_mt (/Users/gauthierroy/.cache/huggingface/datasets/Helsinki-NLP___tatoeba_mt/eng-nld/0.0.0/01e819f3f64a772a2ca70949061d295d3a2dc99d05183fe4776a3be23f75f619)


In [4]:
data_de[0]

{'sourceLang': 'deu',
 'targetlang': 'eng',
 'sourceString': '1960 wurden die Kriegsschäden beseitigt, und das Schloss wurde zu einem Hotel umgebaut.',
 'targetString': 'In 1960, the collateral had been removed and the castle was refurbished into a hotel.'}

In [5]:
data_fr[1]

{'sourceLang': 'eng',
 'targetlang': 'fra',
 'sourceString': 'Aah. Now I understand.',
 'targetString': 'Ah ! Maintenant, je comprends.'}

In [6]:
data_nld[0]

{'sourceLang': 'eng',
 'targetlang': 'nld',
 'sourceString': '2020 was a bad year.',
 'targetString': '2020 was een slecht jaar.'}

In [14]:
x = tokenizer(data_de[0]["sourceString"], return_tensors="pt", padding=True)

set_proba, sequence = model.generate_set(x)

In [15]:
set_proba

tensor([[[3.3309e-05, 1.6649e-06, 2.7034e-06,  ..., 1.5516e-06,
          1.6240e-06, 0.0000e+00],
         [2.1797e-05, 7.7060e-07, 1.9123e-05,  ..., 7.1263e-07,
          7.3666e-07, 0.0000e+00],
         [6.7088e-05, 1.7093e-06, 4.1161e-01,  ..., 1.6444e-06,
          1.6009e-06, 0.0000e+00],
         ...,
         [1.4301e-05, 1.2405e-06, 8.3467e-06,  ..., 1.3257e-06,
          1.3105e-06, 0.0000e+00],
         [4.8149e-04, 1.7255e-06, 2.2016e-03,  ..., 1.7425e-06,
          1.7726e-06, 0.0000e+00],
         [9.0207e-01, 1.7475e-06, 3.2739e-05,  ..., 1.6284e-06,
          1.5849e-06, 0.0000e+00]]])

In [16]:
torch.sum(set_proba,dim=2)

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]])

In [17]:
set_proba.shape

torch.Size([1, 18, 58101])

In [18]:
anomaly_score = model.anomaly_score(x, 'Renyi', 0.05)

18


In [19]:
anomaly_score

tensor([-0.1351])

In [13]:
torch.ones(set_proba.shape).shape

torch.Size([2, 32, 58101])

In [38]:
model.config.vocab_size

58101

In [64]:
if not None:
    print("yo")

yo


In [40]:
tensor_score = torch.stack(list(generated_ids.scores), dim=0).permute(1,0,2)

In [41]:
tensor_score.shape

torch.Size([2, 32, 58101])

In [57]:
softmax = torch.nn.Softmax(dim=2)

In [60]:
tensor_score[0].shape

torch.Size([32, 58101])

In [59]:
torch.sum(softmax(tensor_score)[0],dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [62]:
softmax(tensor_score)[0][31]

tensor([3.8194e-04, 1.5870e-06, 1.6781e-06,  ..., 1.3922e-06, 1.4336e-06,
        0.0000e+00])

In [14]:
tokenizer.batch_decode(generated_ids.sequences, skip_special_tokens=True)

['In 1961 John F. Kennedy told the Americans: "Do not ask what your country can do for you, but what you can do for your country!',
 'In 1960, the war damage was eliminated and the castle was converted into a hotel.']

In [15]:
u = torch.ones(model.config.vocab_size)/model.config.vocab_size

In [16]:
u

tensor([1.7211e-05, 1.7211e-05, 1.7211e-05,  ..., 1.7211e-05, 1.7211e-05,
        1.7211e-05])

In [17]:
torch.sum(u)

tensor(1.0000)

In [18]:
softmax = torch.nn.Softmax()
scores = softmax(generated_ids.scores[0][0])

/var/folders/kf/rsvfsx611_3f7p2ctq98yhqm0000gn/T/ipykernel_2741/3440280699.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  scores = softmax(generated_ids.scores[0][0])


In [20]:
from utils_generator import renyi_divergence

renyi_divergence(u,scores,0.05)

tensor(-0.4272)

In [141]:
type(model)

transformers.models.marian.modeling_marian.MarianMTModel

In [45]:
x = tokenizer(data_nld[0]["targetString"], return_tensors="pt")

generated_ids = model.generate(**x)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

/Users/gauthierroy/opt/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['2020 was een slecht jaar.']

In [42]:
data_fr[1]

{'sourceLang': 'eng',
 'targetlang': 'fra',
 'sourceString': 'Aah. Now I understand.',
 'targetString': 'Ah ! Maintenant, je comprends.'}

In [46]:
generated_ids

tensor([[58100,  4526,    58,   322,    15, 12923, 19988,    46,   990,   637,
             3,     0]])

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds